## Project 1: Language Modeling

In this project, you will implement several different types of language models for text.  We'll start with n-gram models, then move on to neural n-gram and LSTM language models.

**Warning: Do not start this project the day before it is due!**
Some parts require 20 minutes or more to run, so debugging and tuning can take a significant amount of time.

Our dataset for this project will be the WikiText2 language modeling dataset.  We provide some of the basic preprocessing, such as tokenization and rare word filtering (using the `<unk>` token).
Therefore, we can assume that all word types in the val/test set appear at least once in the training set.

In [ ]:
!pip install datasets

In [ ]:
# This block handles some imports and defines some constants.
# You shouldn't need to edit this, but if you want to
# import other standard python packages, that is fine.

# imports
from collections import Counter, defaultdict
import copy
import numpy as np
import math
import tqdm
import random
import pdb
from typing import List, Optional, Tuple, Union

from datasets import load_dataset
import torch
from torch import nn
import torch.nn.functional as F

# Some constants
UNK_TOK = "<unk>"
PAD_TOK = "<pad>"
EOS_TOK = "<eos>"

In [ ]:
# This block defines the Vocabulary class we need later.
# You shouldn't need to edit this.

class Vocab:
    def __init__(self, train_text: List[str], min_freq=0):
        """
        We collect counts from train_text.
        train_text: a list of tokens.
        min_freq: if a token appears strictly less than this, it will not be
            added to vocab.
        """
        special_tokens = [UNK_TOK, PAD_TOK, EOS_TOK]

        counter = Counter(train_text)
        # Note that the order is fixed as long as the training text is the same.
        # it's sorted by frequency.
        all_tokens = [
            t for t, c in counter.most_common()
            if c >= min_freq and t not in special_tokens
        ]

        self.all_tokens = special_tokens + all_tokens
        self.str_to_id = {s: i for i, s in enumerate(self.all_tokens)}

        self.unk_tok = UNK_TOK
        self.pad_tok = PAD_TOK
        self.eos_tok = EOS_TOK

    def size(self) -> int:
        return len(self.all_tokens)


    def ids_to_strs(self, indices: List[int]) -> List[str]:
        return [self.all_tokens[ii] for ii in indices]


    def strs_to_ids(self, strings: List[str]) -> List[int]:
        return [self.str_to_id[s] for s in strings]


    def __contains__(self, token: str) -> bool:
        return token in self.str_to_id

In [ ]:
# This block downloads and processes the data.
# You shouldn't need to edit this.

wikitext2_dataset = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")
print(f"Raw train examples: {wikitext2_dataset['train']['text'][:10]}")

# just use the simplest one for now
tokenizer = lambda x: x.split()

# tokenize datatsets
def preprocess(_dataset: List[str]) -> List[str]:
    """
    Each sentence in _dataset is tokenized into a list of strings.
    _dataset: List[str]. Each string is a sentence.
    """
    ret = []
    for sent in _dataset:
        sent = sent.rstrip('\n')
        # skip empty sentences
        if not sent:
            continue
        # add EOS to the end of sentence
        ret += tokenizer(sent) + [EOS_TOK]
    return ret

tok_train_dataset = preprocess(wikitext2_dataset['train']['text'])
tok_validation_dataset = preprocess(wikitext2_dataset['validation']['text'])
tok_test_dataset = preprocess(wikitext2_dataset['test']['text'])
print(f"Dataset size (#tokens) - Train: {len(tok_train_dataset)}; Validation: {len(tok_validation_dataset)}; Test: {len(tok_test_dataset)}.")

# build vocabulary: use `min_freq` to model UNK in training
### You'll need this vocab throughout this HW.
vocab = Vocab(tok_train_dataset, min_freq=2)
print(f"Vocab size: {vocab.size()}. Examples: {vocab.ids_to_strs(list(range(20)))}")

# handle UNKs properly
def replace_unseen_with_unk(_dataset: List[str]) -> List[str]:
    """
    We replace the unseen tokens in _dataset with vocab.unk_tok.
    """
    new_data = []
    for tok in _dataset:
        if tok in vocab:
            new_data.append(tok)
        else:
            new_data.append(vocab.unk_tok)
    return new_data

### You'll need these three datasets throughout this HW.
tok_train_dataset = replace_unseen_with_unk(tok_train_dataset)
tok_validation_dataset = replace_unseen_with_unk(tok_validation_dataset)
tok_test_dataset = replace_unseen_with_unk(tok_test_dataset)
print(f"Final train examples: {tok_train_dataset[:40]}")
print(f"Final val examples: {tok_validation_dataset[:40]}")

We've implemented a unigram model here as a demonstration.

In [ ]:
class UnigramModel:
    def __init__(self, train_text: List[str]):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)


    def probability(self, word: str) -> float:
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix: List[str]) -> List[str]:
        """
        Return a list of probabilities for each word in the vocabulary.
        In unigram model, `text_prefix` doesn't matter as we are not using any
            context at all.
        """
        return [self.probability(word) for word in vocab.all_tokens]

    def perplexity(self, full_text: List[str]) -> float:
        """Return the perplexity of the model on a text as a float.

        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter
            # as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(tok_train_dataset)
print('unigram validation perplexity:',
      unigram_demonstration_model.perplexity(tok_test_dataset))

unigram validation perplexity: 1057.2131456213988


In [ ]:
def check_validity(model):
    """
    Performs several sanity checks on your model:
      1) That `next_word_probabilities` returns a valid distribution
      2) That perplexity matches a perplexity calculated from `next_word_probabilities`

    Although it is possible to calculate perplexity from `next_word_probabilities`,
      it is still good to have a separate more efficient method that only computes
      the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = tok_validation_dataset[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab.str_to_id[tok_validation_dataset[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob))

    perplexity = math.exp(-np.mean(log_probabilities))
    your_perplexity = model.perplexity(tok_validation_dataset[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


In [ ]:
check_validity(unigram_demonstration_model)

To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [ ]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(vocab.all_tokens, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

# unigram model does not utilize prefix
print(generate_text(unigram_demonstration_model, prefix=""))

lie released down , driving = . that Rifle Margaret 000 or demonstrated Telugu same reduced symbolic five the sound


TODO: Copy the printed output to your report.

In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this recent paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following function will be used to generate the required output files.

In [ ]:
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes_short.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab_short.txt

def save_truncated_distribution(model, filename, short=True):
    """Generate a file of truncated distributions.

    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    vocab_name = 'eval_output_vocab'
    prefixes_name = 'eval_prefixes'

    if short:
      vocab_name += '_short'
      prefixes_name += '_short'

    with open(f'{vocab_name}.txt', 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = sorted(list(set([vocab.str_to_id[s] if s in vocab else vocab.str_to_id[vocab.unk_tok]
                      for s in eval_vocab])))

    all_selected_probabilities = []
    with open(f'{prefixes_name}.txt', 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.notebook.tqdm(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

In [ ]:
save_truncated_distribution(unigram_demonstration_model,
                            'unigram_demonstration_predictions.npy')

  0%|          | 0/1000 [00:00<?, ?it/s]

saved unigram_demonstration_predictions.npy


### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

This is an example of bigram model with smoothing:
$$P(w_2|w_1)=\frac{C(w_1,w_2)+\alpha}{C(w_1)+N\alpha}$$

where $N$ is the vocab size and $C$ is the count for the given unigram/bigram.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One **edge case** you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You may handle this by using a uniform distribution over the vocabulary.

A properly implemented bi-gram model should get a perplexity about/below **635** on the validation set.

**Note**: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass.
Also, we suggest pre-computing and caching the counts $C$ when you initialize `NGramModel` for efficiency.

In [ ]:
from collections import defaultdict
import math
import numpy as np
from typing import List, Tuple

class NGramModel:
    def __init__(self, train_text: List[str], n: int = 2, alpha: float = 3e-3):
        """
        Initializes the model, computes n-gram and (n-1)-gram counts from the training text.
        """
        self.n = n
        self.smoothing = alpha
        self.vocab_size = len(set(train_text))  # Size of the unique vocabulary
        self.train_text = train_text

        # Store n-gram and (n-1)-gram counts
        self.ngram_freq = defaultdict(int)
        self.n_minus_1_freq = defaultdict(int)

        # Count n-grams and (n-1)-grams from the training text
        for i in range(len(train_text) - n + 1):
            ngram = tuple(train_text[i:i + n])
            self.ngram_freq[ngram] += 1

            if n > 1:
                n_minus_1_gram = tuple(train_text[i:i + n - 1])
                self.n_minus_1_freq[n_minus_1_gram] += 1

    def n_gram_probability(self, n_gram: Tuple[str, ...]):
        """
        Returns the smoothed probability of the n-gram.
        """
        if self.n == 1:
            unigram = (n_gram[-1],)
            unigram_count = self.ngram_freq[unigram]
            total_unigrams = sum(self.ngram_freq.values())
            prob = (unigram_count + self.smoothing) / (total_unigrams + self.smoothing * self.vocab_size)
            return prob

        # For n-grams (n > 1)
        assert len(n_gram) == self.n, f"Expected {self.n}-gram, got {n_gram}"

        n_minus_1_gram = n_gram[:-1]
        ngram_count = self.ngram_freq[n_gram]
        n_minus_1_gram_count = self.n_minus_1_freq[n_minus_1_gram]

        # Apply smoothing
        prob = (ngram_count + self.smoothing) / (n_minus_1_gram_count + self.smoothing * self.vocab_size)
        return prob

    def next_word_probabilities(self, text_prefix: List[str]):
        """
        Returns a list of probabilities for each word in the vocabulary given the text prefix.
        """
        if len(text_prefix) < self.n - 1:
            # If the prefix is shorter than expected, return uniform probabilities
            return [1 / self.vocab_size] * self.vocab_size

        # Use the last (n-1) tokens from the prefix for context
        n_minus_1_gram = tuple(text_prefix[-(self.n - 1):])

        # Compute probabilities for each word in the vocabulary
        probabilities = []
        for word in vocab.all_tokens:
            ngram = n_minus_1_gram + (word,)
            probabilities.append(self.n_gram_probability(ngram))

        return probabilities

    def perplexity(self, full_text: List[str]):
        """
        Returns the perplexity for the given text.
        """
        log_probs = []

        # Loop through the text to calculate log probabilities
        for i in range(len(full_text) - self.n + 1):
            ngram = tuple(full_text[i:i + self.n])
            prob = self.n_gram_probability(ngram)
            log_probs.append(math.log(prob, 2))

        # Handle the first n-1 tokens by assuming a uniform distribution
        uniform_log_prob = math.log(1 / self.vocab_size, 2)
        log_probs = [uniform_log_prob] * (self.n - 1) + log_probs

        # Compute perplexity
        return 2 ** (-np.mean(log_probs))



# Example usage:
unigram_model = NGramModel(tok_train_dataset, n=1)  # Equivalent to a Unigram model
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(tok_validation_dataset))

bigram_model = NGramModel(tok_train_dataset, n=2)  # Bigram model
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(tok_validation_dataset))

trigram_model = NGramModel(tok_train_dataset, n=3)  # Trigram model
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(tok_validation_dataset))


unigram validation perplexity: 1096.261759471866
bigram validation perplexity: 635.624422688812
trigram validation perplexity: 4287.867599767591


In [ ]:
save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes

  0%|          | 0/1000 [00:00<?, ?it/s]

saved bigram_predictions.npy


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.  (We will use this to help us calibrate our scoring on the test set.)

TODO: Report the perplexity in your report.

Bigram validation perplexity: **635.624422688812**

We can also generate samples from the model to get an idea of how it is doing.

In [ ]:
print(generate_text(bigram_model))

<eos> <eos> = = <eos> = Service stead Tamar conspicuous Pune Marj braces boy JGT Politically Sordi Saddleworth Get You To satyagraha


We now free up some RAM, **it is important to run the cell below, otherwise you will likely run out of RAM in the Colab runtime.**

In [ ]:
# Free up some RAM.
del bigram_model
del trigram_model

This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|w_{i-n+1}^{i-1}\right)=\frac{max\left\{C(w_{i-n+1}^i)-\delta,0\right\}}{\sum_{w_i} C(w_{i-n+1}^i)} + \alpha(w_{i-n+1}^{i-1}) P(w_i|w_{i-n+2}^{i-1})$$

$$\alpha\left(w_{i-n+1}^{i-1}\right)=\frac{\delta N_{1+}(w_{i-n+1}^{i-1})}{{\sum_{w_i} C(w_{i-n+1}^i)}}$$
where $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation).  If $\sum_{w_i} C(w_{i-n+1}^i)=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $\delta$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity around/below **310**.

In [ ]:
from collections import defaultdict
from typing import List, Tuple, Union

class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text: List[str],
                 lower_order_model: Union[NGramModel, "DiscountBackoffModel"],
                 n: int = 2,
                 delta: float = 0.9):
        """We only use n >= 2 for backoff"""
        super().__init__(train_text, n=n)
        assert n >= 2, "N must be 2 or greater for backoff"

        self.lower_order_model = lower_order_model
        self.discount = delta

        # Initialize structures to store counts
        self.context_counts = defaultdict(int)
        self.n_gram_counts = defaultdict(lambda: defaultdict(int))
        self.N1_plus = defaultdict(int)

        # Precompute counts for n-grams and (n-1)-gram contexts
        self.initialize_counts(train_text)

    def initialize_counts(self, train_text: List[str]):
        """Compute counts of n-grams, contexts, and distinct word counts."""
        for idx in range(len(train_text) - self.n + 1):
            # Extract (n-1)-gram context and the nth word
            context = tuple(train_text[idx:idx+self.n-1])
            word = train_text[idx + self.n - 1]

            # Update context and n-gram counts
            self.context_counts[context] += 1
            self.n_gram_counts[context][word] += 1

            # Track the number of distinct words following each context
            if self.n_gram_counts[context][word] == 1:
                self.N1_plus[context] += 1

    def n_gram_probability(self, n_gram: Tuple[str, ...]) -> float:
        """Calculate the probability of the last word in an n-gram using backoff."""
        context = n_gram[:-1]
        word = n_gram[-1]

        # If context has no counts, back off to the lower-order model
        if context not in self.context_counts or self.context_counts[context] == 0:
            return self.lower_order_model.n_gram_probability(n_gram[1:])

        # Get counts for the context and the current n-gram
        context_count = self.context_counts[context]
        word_count = self.n_gram_counts[context].get(word, 0)

        # Apply discount to the word count
        discounted_count = max(word_count - self.discount, 0)
        discounted_prob = discounted_count / context_count

        # Calculate backoff weight (alpha)
        alpha = (self.discount * self.N1_plus[context]) / context_count

        # Get the backoff probability from the lower-order model
        backoff_prob = self.lower_order_model.n_gram_probability(n_gram[1:])

        # Combine discounted and backoff probabilities
        final_prob = discounted_prob + alpha * backoff_prob

        # Ensure probability is bounded between 0 and 1
        return max(0, min(1, final_prob))

bigram_backoff_model = DiscountBackoffModel(tok_train_dataset, unigram_model, 2)
check_validity(bigram_backoff_model)
print('bigram backoff validation perplexity:', bigram_backoff_model.perplexity(tok_validation_dataset))

trigram_backoff_model = DiscountBackoffModel(tok_train_dataset, bigram_backoff_model, 3)
check_validity(trigram_backoff_model)
print('trigram backoff validation perplexity:', trigram_backoff_model.perplexity(tok_validation_dataset))


bigram backoff validation perplexity: 346.44721606199545
trigram backoff validation perplexity: 310.7262767493293


In [ ]:
save_truncated_distribution(trigram_backoff_model, 'trigram_backoff_predictions.npy') # this might take a few minutes

  0%|          | 0/1000 [00:00<?, ?it/s]

saved trigram_backoff_predictions.npy


TODO: Report your trigram backoff model perplexity.

Trigram backoff validation perplexity: **310.7262767493293**

Free up RAM.

In [ ]:
# Release models we don't need any more.
del unigram_model
del bigram_backoff_model
del trigram_backoff_model

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity around/below **240**.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer (ie. 4 layers total).
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with `nn.Linear`, then using `F.embedding` with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 after first 2 hidden layers. **Note: You will likely find a performance drop if you add a nonlinear activation function after the dimension reduction layer.**
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
* do early stopping based on validation set perplexity.


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, though.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from typing import List
import tqdm


# Dataset class to handle N-Gram input/output pairs
class NeuralNgramDataset(Dataset):
    def __init__(self, text_token_ids: List[int], n: int):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i: int):
        # Handling cases where index is less than n-1
        if i < self.n - 1:
            padding = [vocab.str_to_id[vocab.eos_tok]] * (self.n - i - 1)
            prev_token_ids = padding + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i - self.n + 1:i]

        assert len(prev_token_ids) == self.n - 1, prev_token_ids

        x = torch.tensor(prev_token_ids, dtype=torch.long)
        y = torch.tensor(self.text_token_ids[i], dtype=torch.long)
        return x, y


# Recurrent Neural Network-based N-Gram Model
class NeuralNGramNetwork(nn.Module):
    def __init__(self, n: int, embed_dim: int = 128, hidden_dim: int = 512, dropout_rate: float = 0.1):
        super(NeuralNGramNetwork, self).__init__()
        self.n = n
        vocab_size = len(vocab.str_to_id)

        # Embedding layer
        self.embedding_layer = nn.Embedding(vocab_size, embed_dim)

        # Using GRU for sequential modeling
        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers=2, batch_first=True, dropout=dropout_rate)

        # Layer to project GRU hidden states to embedding dimension
        self.hidden_to_embedding = nn.Linear(hidden_dim, embed_dim)

        # Output layer for classification
        self.output_layer = nn.Linear(embed_dim, vocab_size)

        # Weight tying to share the embedding weights with output layer
        self.output_layer.weight = self.embedding_layer.weight

    def forward(self, x):
        embedded_seq = self.embedding_layer(x)  # Shape: (batch_size, n-1, embed_dim)

        # Forward pass through the GRU layer
        rnn_output, _ = self.gru(embedded_seq)

        # Take the last hidden state of GRU
        final_hidden_state = rnn_output[:, -1, :]  # Shape: (batch_size, hidden_dim)

        # Project to embedding dimension
        projected_output = self.hidden_to_embedding(final_hidden_state)  # Shape: (batch_size, embed_dim)

        # Calculate logits
        logits = self.output_layer(projected_output)  # Shape: (batch_size, vocab_size)

        return logits


# Wrapper class for training and evaluation
class NeuralNGramModel:
    def __init__(self, n: int, device: str = "cpu", **model_configs):
        self.n = n
        self.device = device
        vocab_size = len(vocab.str_to_id)

        if "cuda" in self.device:
            assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

        self.network = NeuralNGramNetwork(n, vocab_size, **model_configs).to(self.device)

    # Modified training loop with some structure and handling
    def train(self, n_epoch: int = 10, lr: float = 0.001, batch_size: int = 128, grad_clip: float = 0.5, weight_decay: float = 1e-5):
        # Dataset and DataLoader creation
        train_dataset = NeuralNgramDataset(vocab.strs_to_ids(tok_train_dataset), self.n_gram)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Optimizer and scheduler setup
        optimizer = torch.optim.Adam(self.network.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)
        loss_function = nn.CrossEntropyLoss()

        # Training loop
        for epoch in range(n_epoch):
            epoch_loss = 0
            self.network.train()

            # Loop over batches
            for x_batch, y_batch in tqdm.notebook.tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
                x_batch, y_batch = x_batch.to(self.device), y_batch.to(self.device)

                # Zero gradients, forward pass, compute loss, and backpropagate
                optimizer.zero_grad()
                logits = self.network(x_batch)
                loss = loss_function(logits, y_batch)
                loss.backward()

                # Clip gradients to prevent exploding gradients
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), grad_clip)
                optimizer.step()

                epoch_loss += loss.item()

            # Scheduler step based on epoch loss
            scheduler.step(epoch_loss / len(train_dataloader))

            print(f"Epoch {epoch + 1}/{n_epoch}, Loss: {epoch_loss / len(train_dataloader):.4f}")

    # Generate next word probabilities based on a given text prefix
    def next_word_probabilities(self, text_prefix: List[str]) -> List[float]:
        self.network.eval()

        # Convert input text prefix to token IDs
        token_ids = vocab.strs_to_ids(text_prefix)

        # Pad the sequence if it's shorter than expected
        if len(token_ids) < self.n_gram - 1:
            pad = [vocab.str_to_id[vocab.eos_tok]] * (self.n_gram - 1 - len(token_ids))
            token_ids = pad + token_ids

        # Convert token IDs to tensor and pass through the model
        input_tensor = torch.tensor(token_ids, dtype=torch.long).unsqueeze(0).to(self.device)

        with torch.no_grad():
            logits = self.network(input_tensor)

        # Convert logits to probabilities using softmax
        probabilities = F.softmax(logits, dim=-1).squeeze(0).tolist()
        return probabilities

    # Perplexity evaluation method
    def perplexity(self, text: List[str]) -> float:
        self.network.eval()

        # Create dataset and data loader for the text
        test_dataset = NeuralNgramDataset(vocab.strs_to_ids(text), self.n_gram)
        test_loader = DataLoader(test_dataset, batch_size=256)

        total_loss = 0
        criterion = nn.CrossEntropyLoss()

        # Evaluate model on validation data
        with torch.no_grad():
            for x_batch, y_batch in test_loader:
                x_batch, y_batch = x_batch.to(self.device), y_batch.to(self.device)

                logits = self.network(x_batch)
                loss = criterion(logits, y_batch)

                total_loss += loss.item()

        # Calculate average loss and convert to perplexity
        avg_loss = total_loss / len(test_loader)
        perplexity = torch.exp(torch.tensor(avg_loss))
        return perplexity.item()



In [ ]:
# Initialize the model for a trigram setup
neural_trigram_model = NeuralNGramModel(3,device="cuda")
neural_trigram_model.train(lr=5e-4)
# Calculate and print perplexity on the validation set
print('Neural trigram validation perplexity:', neural_trigram_model.perplexity(tok_validation_dataset))

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1:   0%|          | 0/8109 [00:00<?, ?it/s]

Epoch 1/10, Loss: 6.9401


Epoch 2:   0%|          | 0/8109 [00:00<?, ?it/s]

Epoch 2/10, Loss: 6.1569


...

Epoch 8/10, Loss: 5.3889


Epoch 9:   0%|          | 0/8109 [00:00<?, ?it/s]

Epoch 9/10, Loss: 5.3325


Epoch 10:   0%|          | 0/8109 [00:00<?, ?it/s]

Epoch 10/10, Loss: 5.2824
Neural trigram validation perplexity: 237.67050170898438


In [ ]:
save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy', short=False)

  0%|          | 0/5000 [00:00<?, ?it/s]

saved neural_trigram_predictions.npy


TODO: Fill in your neural trigram perplexity in the report.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

*   List item
*   List item



Neural trigram validation perplexity: 237.67050170898438

Free up RAM.

In [ ]:
# Delete model we don't need.
del neural_trigram_model

### LSTM Model

For this stage of the project, you will implement an LSTM language model.

For recurrent language modeling, the data batching strategy is a bit different from what is used in some other tasks.  Sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch.
To properly deal with this input format, you should **save the last state of the LSTM from a batch to feed in as the first state of the next batch**.  When you save state across different batches, you should call `.detach()` on the state tensors before the next batch to tell PyTorch not to backpropagate gradients through the state into the batch you have already finished (which will cause a runtime error).

We expect your model to reach a validation perplexity around/below **214**.
The following architecture and hyperparameters should be sufficient to get there.
* 3 LSTM layers with 512 units
* dropout of 0.5 after each LSTM layer
* instead of projecting directly from the last LSTM output to the vocabulary size for softmax, project down to a smaller size first (e.g. 512->128->vocab_size). **NOTE: You may find that adding nonlinearities between these layers can hurt performance, try without first.**
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128
* train with Adam (using default learning rates) for at least 20 epochs


In [ ]:
# ref: https://github.com/pytorch/text/blob/0.5.0/torchtext/data/iterator.py#L173

class LstmDataIterator:
    def __init__(self, dataset: List[int], batch_size: int = 64, seq_len: int = 32, device: str = "cpu"):
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.device = device

        # pad the dataset so that it is divisible by batch_size
        dataset = dataset + [vocab.str_to_id[vocab.pad_tok]] * (math.ceil(len(dataset) / batch_size) * batch_size - len(dataset))

        self.n_samples = math.ceil(
            (len(dataset) // batch_size - 1) / seq_len
        )

        dataset = torch.tensor(dataset, dtype=torch.long)
        self.dataset = dataset.view(batch_size, -1).t().contiguous()

    def __len__(self):
        return self.n_samples

    def __getitem__(self, i: int):
        start = i * self.seq_len
        end = min(start + self.seq_len, self.dataset.shape[0] - 1)

        inputs = self.dataset[start : end]
        outputs = self.dataset[start + 1 : end + 1]
        assert inputs.shape == outputs.shape, f"{i}: {inputs.shape} {outputs.shape}"
        # (seq_len, batch_size)
        return inputs.to(self.device), outputs.to(self.device)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from typing import List, Optional, Tuple

class LSTMNetwork(nn.Module):
    # A PyTorch Module that holds the neural network for your model

    def __init__(self, embed_dim: int = 128, n_layer: int = 3, hidden_dim: int = 512, dropout_rate: float = 0.5):
        super().__init__()

        vocab_size = len(vocab.str_to_id)

        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)

        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=n_layer,
            dropout=dropout_rate,
        )

        # Layer normalization for stabilization
        self.layer_norm = nn.LayerNorm(hidden_dim)

        # Projection layers
        self.proj = nn.Linear(hidden_dim, embed_dim)

        # Output layer (tied weights with embedding layer)
        self.output_layer = nn.Linear(embed_dim, vocab_size)
        self.output_layer.weight = self.embedding.weight  # Weight tying

        # Weight initialization
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

    def forward(self, x: torch.Tensor, state: Optional[Tuple[torch.Tensor, torch.Tensor]] = None):
        """Compute the output of the network."""
        embedded = self.embedding(x)  # (seq_len, batch_size, embed_dim)

        # Pass through LSTM layers
        lstm_out, state = self.lstm(embedded, state)  # lstm_out: (seq_len, batch_size, hidden_dim)

        # Apply layer normalization
        lstm_out = self.layer_norm(lstm_out)

        # Project to embedding dimension
        proj_out = self.proj(lstm_out)  # (seq_len, batch_size, embed_dim)

        # Compute logits
        logits = self.output_layer(proj_out)  # (seq_len, batch_size, vocab_size)

        # Compute log probabilities
        log_probs = F.log_softmax(logits, dim=-1)

        return log_probs, state


class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."

    def __init__(self, device: str = "cpu", **model_configs):
        self.device = device
        if "cuda" in self.device:
            assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

        self.network = LSTMNetwork(**model_configs).to(self.device)

    def train(
        self,
        n_epoch: int = 20, lr: float = 1e-3, batch_size: int = 64, seq_len: int = 32
    ):
        train_data_iter = LstmDataIterator(vocab.strs_to_ids(tok_train_dataset), batch_size, seq_len, self.device)

        optimizer = torch.optim.AdamW(self.network.parameters(), lr=lr, weight_decay=1e-5)
        criterion = nn.NLLLoss(ignore_index=vocab.str_to_id[vocab.pad_tok])

        self.network.train()

        for epoch in range(n_epoch):
            hidden_state = None  # Initialize hidden state at the beginning of each epoch
            total_loss = 0.0

            for i in range(len(train_data_iter)):
                inputs, targets = train_data_iter[i]  # (seq_len, batch_size)
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                optimizer.zero_grad()

                # Forward pass
                log_probs, hidden_state = self.network(inputs, hidden_state)

                # Reshape for loss computation
                loss = criterion(
                    log_probs.view(-1, log_probs.size(2)),  # (seq_len * batch_size, vocab_size)
                    targets.view(-1),  # (seq_len * batch_size)
                )

                # Backward pass and optimization
                loss.backward()

                # Gradient clipping to avoid exploding gradients
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), max_norm=1)

                optimizer.step()

                total_loss += loss.item()

                # Detach hidden state to prevent backprop through entire sequence
                hidden_state = tuple(h.detach() for h in hidden_state)

            avg_loss = total_loss / len(train_data_iter)
            print(f"Epoch {epoch + 1}/{n_epoch}, Loss: {avg_loss:.4f}")

    def next_word_probabilities(self, text_prefix: List[str]):
        "Return a list of probabilities for each word in the vocabulary."
        self.network.eval()
        with torch.no_grad():
            ids_prefix = torch.tensor(
                vocab.strs_to_ids(text_prefix), dtype=torch.long, device=self.device
            ).view(-1, 1)  # (seq_len, batch_size=1)

            # Initialize hidden state
            hidden_state = None

            # Forward pass
            log_probs, hidden_state = self.network(ids_prefix, hidden_state)

            # Get the last timestep's log probabilities
            last_log_probs = log_probs[-1, 0, :]  # (vocab_size,)

            # Convert to probabilities
            probabilities = last_log_probs.exp().cpu().numpy()
            return probabilities.tolist()

    def dataset_perplexity(self, dataset: List[str], batch_size: int = 64, seq_len: int = 32):
        "Return perplexity as a float."
        self.network.eval()
        data_iterator = LstmDataIterator(
            vocab.strs_to_ids(dataset), batch_size, seq_len, self.device
        )

        total_loss = 0.0
        total_tokens = 0
        criterion = nn.NLLLoss(
            ignore_index=vocab.str_to_id[vocab.pad_tok], reduction="sum"
        )

        with torch.no_grad():
            hidden_state = None
            for i in range(len(data_iterator)):
                inputs, targets = data_iterator[i]
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                # Forward pass
                log_probs, hidden_state = self.network(inputs, hidden_state)

                # Compute loss
                loss = criterion(
                    log_probs.view(-1, log_probs.size(2)),
                    targets.view(-1),
                )

                total_loss += loss.item()

                # Count non-padding tokens
                non_pad_mask = targets.view(-1) != vocab.str_to_id[vocab.pad_tok]
                total_tokens += non_pad_mask.sum().item()

                # Detach hidden state
                hidden_state = tuple(h.detach() for h in hidden_state)

        # Calculate perplexity
        avg_loss = total_loss / total_tokens
        perplexity = math.exp(avg_loss)
        return perplexity


In [ ]:
lstm_model = LSTMModel(device="cuda")
lstm_model.train()

print('lstm validation perplexity:', lstm_model.dataset_perplexity(tok_validation_dataset))

Epoch 1/20, Loss: 7.6759
Epoch 2/20, Loss: 6.5856
Epoch 3/20, Loss: 6.3243
Epoch 4/20, Loss: 6.1267
Epoch 5/20, Loss: 5.9621
Epoch 6/20, Loss: 5.8222
Epoch 7/20, Loss: 5.6997
Epoch 8/20, Loss: 5.5922
Epoch 9/20, Loss: 5.4949
Epoch 10/20, Loss: 5.4072
Epoch 11/20, Loss: 5.3267
Epoch 12/20, Loss: 5.2530
Epoch 13/20, Loss: 5.1855
Epoch 14/20, Loss: 5.1242
Epoch 15/20, Loss: 5.0669
Epoch 16/20, Loss: 5.0131
Epoch 17/20, Loss: 4.9616
Epoch 18/20, Loss: 4.9133
Epoch 19/20, Loss: 4.8631
Epoch 20/20, Loss: 4.8142
lstm validation perplexity: 166.70655249864177


In [ ]:
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

  0%|          | 0/5000 [00:00<?, ?it/s]

saved lstm_predictions.npy


TODO: Report your LSTM perplexity.

LSTM validation perplexity: 166.70655249864177

# Experimentation: 1-Page Report

Now it's time for you to experiment.  Try to reach a validation perplexity below 120. You may either modify the LSTM class above, or copy it down to the code cell below and modify it there. Just **be sure to run code cell below to generate results with your improved LSTM**.  

It is okay if the bulk of your improvements are due to hyperparameter tuning (such as changing number or sizes of layers), but implement at least one more substantial change to the model.  Here are some ideas (several of which come from https://arxiv.org/pdf/1708.02182.pdf):
* activation regularization - add a l2 regularization penalty on the activation of the LSTM output (standard l2 regularization is on the weights)
* weight-drop regularization - apply dropout to the weight matrices instead of activations
* learning rate scheduling - decrease the learning rate during training
* embedding dropout - zero out the entire embedding for a random set of words in the embedding matrix
* ensembling - average the predictions of several models trained with different initialization random seeds
* temporal activation regularization - add l2 regularization on the difference between the LSTM output activations at adjacent timesteps

You may notice that most of these suggestions are regularization techniques.  This dataset is considered fairly small, so regularization is one of the best ways to improve performance.

TODO: In the report, submit a write-up describing the extensions and/or modifications that you tried.  Your description should be **1-page maximum** in length.
For full credit, your write-up should include:
1.   A concise and precise description of the extension that you tried.
2.   A motivation for why you believed this approach might improve your model.
3.   A discussion of whether the extension was effective and/or an analysis of the results.  This will generally involve some combination of tables, learning curves, etc.
4.   A bottom-line summary of your results comparing validation perplexities of your improvement to the original LSTM.


Run the cell below in order to train your improved LSTM and evaluate it.  

In [ ]:
# ref: https://github.com/pytorch/text/blob/0.5.0/torchtext/data/iterator.py#L173

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math
from typing import List, Optional, Tuple

class LstmDataIterator:
    def __init__(self, dataset: List[int], batch_size: int = 64, seq_len: int = 32, device: str = "cpu"):
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.device = device

        # pad the dataset so that it is divisible by batch_size
        dataset = dataset + [vocab.str_to_id[vocab.pad_tok]] * (math.ceil(len(dataset) / batch_size) * batch_size - len(dataset))

        self.n_samples = math.ceil(
            (len(dataset) // batch_size - 1) / seq_len
        )

        dataset = torch.tensor(dataset, dtype=torch.long)
        self.dataset = dataset.view(batch_size, -1).t().contiguous()

    def __len__(self):
        return self.n_samples

    def __getitem__(self, i: int):
        start = i * self.seq_len
        end = min(start + self.seq_len, self.dataset.shape[0] - 1)

        inputs = self.dataset[start : end]
        outputs = self.dataset[start + 1 : end + 1]
        assert inputs.shape == outputs.shape, f"{i}: {inputs.shape} {outputs.shape}"
        # (seq_len, batch_size)
        return inputs.to(self.device), outputs.to(self.device)


class LSTMNetwork(nn.Module):
    def __init__(self, embed_dim: int = 128, n_layer: int = 3, hidden_dim: int = 512, dropout_rate: float = 0.5):
        super().__init__()

        vocab_size = len(vocab.str_to_id)

        # Embedding layer with dropout for regularization
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        self.embed_dropout = nn.Dropout(dropout_rate)

        # LSTM layer with weight dropout regularization
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=n_layer,
            dropout=0,  # Dropout will be applied to weights
        )
        self.weight_dropout = nn.Dropout(dropout_rate)  # Dropout applied to weights in the LSTM

        # Projection layer
        self.proj = nn.Linear(hidden_dim, embed_dim)

        # Output layer (tied weights with embedding layer)
        self.output_layer = nn.Linear(embed_dim, vocab_size)
        self.output_layer.weight = self.embedding.weight  # Weight tying

    def forward(self, x: torch.Tensor, state: Optional[Tuple[torch.Tensor, torch.Tensor]] = None):
        # Apply embedding dropout
        embedded = self.embed_dropout(self.embedding(x))  # (seq_len, batch_size, embed_dim)

        # Forward pass through LSTM
        lstm_out, state = self.lstm(embedded, state)  # (seq_len, batch_size, hidden_dim)

        # Apply weight dropout regularization
        lstm_out = self.weight_dropout(lstm_out)

        # Project to embedding dimension
        proj_out = self.proj(lstm_out)  # (seq_len, batch_size, embed_dim)

        # Compute logits
        logits = self.output_layer(proj_out)  # (seq_len, batch_size, vocab_size)

        # Compute log probabilities
        log_probs = F.log_softmax(logits, dim=-1)

        return log_probs, state


class LSTMModel:
    def __init__(self, device: str = "cpu", **model_configs):
        self.device = device
        if "cuda" in self.device:
            assert torch.cuda.is_available(), "No GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

        self.network = LSTMNetwork(**model_configs).to(self.device)
        self.scheduler = ReduceLROnPlateau(optimizer=optim.Adam(self.network.parameters(), lr=1e-3),
                                           mode='min', factor=0.5, patience=2, verbose=True)

    def train(self, n_epoch: int = 20, lr: float = 1e-3, batch_size: int = 64, seq_len: int = 32, grad_clip: float = 5.0):
        train_data_iter = LstmDataIterator(vocab.strs_to_ids(tok_train_dataset), batch_size, seq_len, self.device)
        optimizer = optim.Adam(self.network.parameters(), lr=lr)
        criterion = nn.NLLLoss(ignore_index=vocab.str_to_id[vocab.pad_tok])

        for epoch in range(n_epoch):
            hidden_state = None  # Initialize hidden state at the beginning of each epoch
            total_loss = 0.0

            self.network.train()

            for i in range(len(train_data_iter)):
                inputs, targets = train_data_iter[i]
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                optimizer.zero_grad()

                # Forward pass
                log_probs, hidden_state = self.network(inputs, hidden_state)

                # Reshape for loss computation
                loss = criterion(
                    log_probs.view(-1, log_probs.size(2)),  # (seq_len * batch_size, vocab_size)
                    targets.view(-1),  # (seq_len * batch_size)
                )

                # Backward pass and optimization
                loss.backward()

                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), grad_clip)

                optimizer.step()

                total_loss += loss.item()

                # Detach hidden state to prevent backprop through entire sequence
                hidden_state = tuple(h.detach() for h in hidden_state)

            avg_loss = total_loss / len(train_data_iter)
            print(f"Epoch {epoch + 1}/{n_epoch}, Loss: {avg_loss:.4f}")

            # Step the learning rate scheduler
            self.scheduler.step(avg_loss)

    def next_word_probabilities(self, text_prefix: List[str]):
        "Return a list of probabilities for each word in the vocabulary."
        self.network.eval()
        with torch.no_grad():
            ids_prefix = torch.tensor(
                vocab.strs_to_ids(text_prefix), dtype=torch.long, device=self.device
            ).view(-1, 1)  # (seq_len, batch_size=1)

            # Initialize hidden state
            hidden_state = None

            # Forward pass
            log_probs, hidden_state = self.network(ids_prefix, hidden_state)

            # Get the last timestep's log probabilities
            last_log_probs = log_probs[-1, 0, :]  # (vocab_size,)

            # Convert to probabilities
            probabilities = last_log_probs.exp().cpu().numpy()
            return probabilities.tolist()

    def dataset_perplexity(self, dataset: List[str], batch_size: int = 64, seq_len: int = 32):
        "Return perplexity as a float."
        self.network.eval()
        data_iterator = LstmDataIterator(vocab.strs_to_ids(dataset), batch_size, seq_len, self.device)
        criterion = nn.NLLLoss(ignore_index=vocab.str_to_id[vocab.pad_tok], reduction="sum")

        total_loss = 0.0
        total_tokens = 0

        with torch.no_grad():
            hidden_state = None
            for i in range(len(data_iterator)):
                inputs, targets = data_iterator[i]
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                # Forward pass
                log_probs, hidden_state = self.network(inputs, hidden_state)

                # Compute loss
                loss = criterion(
                    log_probs.view(-1, log_probs.size(2)),
                    targets.view(-1),
                )

                total_loss += loss.item()

                # Count non-padding tokens
                non_pad_mask = targets.view(-1) != vocab.str_to_id[vocab.pad_tok]
                total_tokens += non_pad_mask.sum().item()

                # Detach hidden state
                hidden_state = tuple(h.detach() for h in hidden_state)

        # Calculate perplexity
        avg_loss = total_loss / total_tokens
        perplexity = math.exp(avg_loss)
        return perplexity


In [ ]:
## Feel free to copy your original LSTM solution down here to modify for your report if you'd like.
# YOUR CODE [optionally] HERE
##

lstm_model = LSTMModel(device="cuda")
lstm_model.train()

print('lstm validation perplexity:', lstm_model.dataset_perplexity(tok_validation_dataset))


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20, Loss: 7.0570
Epoch 2/20, Loss: 6.3370
Epoch 3/20, Loss: 6.0638
Epoch 4/20, Loss: 5.8641
Epoch 5/20, Loss: 5.7052
Epoch 6/20, Loss: 5.5765
Epoch 7/20, Loss: 5.4707
Epoch 8/20, Loss: 5.3772
Epoch 9/20, Loss: 5.2982
Epoch 10/20, Loss: 5.2273
Epoch 11/20, Loss: 5.1659
Epoch 12/20, Loss: 5.1089
Epoch 13/20, Loss: 5.0570
Epoch 14/20, Loss: 5.0090
Epoch 15/20, Loss: 4.9656
Epoch 16/20, Loss: 4.9248
Epoch 17/20, Loss: 4.8861
Epoch 18/20, Loss: 4.8502
Epoch 19/20, Loss: 4.8164
Epoch 20/20, Loss: 4.7848
lstm validation perplexity: 144.3142732671605


In [ ]:
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

  0%|          | 0/5000 [00:00<?, ?it/s]

saved lstm_predictions.npy


### Submission

Upload a submission with the following files to Gradescope:
* proj_1.ipynb (rename to match this exactly)
* lstm_predictions.npy (this should also include all improvements from your exploration)
* neural_trigram_predictions.npy
* bigram_predictions.npy
* report.pdf

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a completely different scale from your validation set perplexities due to truncating the distribution and selecting different text.  Don't worry if the values seem much worse.